In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import warnings

from numpy.random import random_sample
from scipy.interpolate import griddata

from lib_geos import gridvec
from lib_seis import Bkspline

warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = 8, 5
plt.rcParams.update({'font.size': 9})

# This is a plotting test function for Bkspline(), which returns a
# spherical spline basis function at specifited lat-lon points.
# It also returns the spatial derivatives of the basis function,
# which are useful for representating derivatives of target functions,
# as well as for damping.
#
# Carl Tape, Amanda McPherson

In [ ]:
# Set up constants, and create sample data

deg = 180/np.pi

# create sample data
numx = 200
ax1 = [-122, -114, 32, 37]
lonmin = ax1[0] 
lonmax = ax1[1]
latmin = ax1[2]
latmax = ax1[3]
lon, lat, numy, X, Y = gridvec(lonmin,lonmax,numx,latmin,latmax)

# select sample spline (DEFAULT: pick one at random)
q = 6      # KEY: determines the scalelength of the spline (q = 0-10)
clon = (lonmax - lonmin)*random_sample() + lonmin
clat = (latmax - latmin)*random_sample() + latmin

print('example spherical spline function defined by:')
print('(clon,clat) = (',clon,',',clat,')')
print('q = ',q)

In [ ]:
# evaluate spline function
ff = Bkspline(clon, clat, q, lon, lat)

In [ ]:
# Plot spline function on the grid

#X, Y, Z = griddataXB(lon,lat,ff[:,0],100,'cubic')
Z = np.reshape(ff[:,0],(numy,numx))

plt.figure()
plt.pcolor(X,Y,Z,shading='nearest',vmin=0,vmax=1)
plt.axis(ax1)
plt.colorbar()
plt.xlabel('Longitude (deg)')
plt.ylabel('Latitude (deg)')
plt.title('Spherical spline basis function, order q=%i, centered at lon=%.2f, lat=%.2f'% (q,clon,clat))
plt.show()

In [ ]:
# plot other scalar fields related to the spline function

ncol = 5
ff = Bkspline(clon, clat, q, lon, lat, ncol=ncol)

dfdp = ff[:,1]
dfdt = ff[:,2]
th   = (90-lat)/deg

# check the computation return Bkspline
dfmag = np.sqrt( dfdt**2 + ((1/np.sin(th).flatten()) * dfdp)**2 )
print('checking the mag:',np.linalg.norm( dfmag - ff[:,4] ))

d1max = np.amax([ np.amax(abs(dfdp)), np.amax(abs(dfdt)) ])

# plotting
stitd = ['f','d f / d φ','d f / d θ','∇^2 f','| ∇ f | ']
plt.figure(figsize=(10,10))

for ii in range(len(stitd)):
    #X, Y, Z = griddataXB(lon,lat,ff[:,ii],100,'cubic')
    Z = np.reshape(ff[:,ii],(numy,numx))
    plt.subplot(3,2,ii+1)
    if ii == 1 or ii == 2:
        plt.pcolor(X,Y,Z,shading='nearest',vmin=-d1max,vmax=d1max)
    else:
        plt.pcolor(X,Y,Z,shading='nearest')
        
    plt.title(stitd[ii])
    plt.axis(ax1)
    plt.colorbar()
    
plt.subplots_adjust(hspace=0.3)    
plt.show()

In [ ]:
# plot the surface gradient

# create sample data
numx2 = 20
lon2, lat2, numy2, X2, Y2 = gridvec(lonmin,lonmax,numx2,latmin,latmax)
ff2 = Bkspline(clon, clat, q, lon2, lat2, ncol=ncol)
Z2 = np.reshape(ff2[:,4],(numy2,numx2))
#X, Y, Z = griddataXB(lon,lat,ff[:,4],100,'cubic')

plt.figure()
plt.pcolor(X2,Y2,Z2,shading='nearest')
# minus sign converts from south component to north component
plt.quiver(lon2,lat2,ff2[:,1],-ff2[:,2],color='k')
plt.title('surface gradient vector field, along with the magnitude')
plt.axis(ax1)
plt.colorbar()
plt.show()

In [ ]:
# WRITE CODE BELOW TO USE Bkspline (you do not need anything from above)
